<a href="https://colab.research.google.com/github/ShlokM08/CSE508_Winter2024_A2_2021421./blob/main/IR_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#IMPORT DRIVE
from google.colab import drive
drive.mount('/content/drive')
!pip install python-docx nltk


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import requests
import pickle

from io import BytesIO
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from PIL import Image

# Load the CSV file
csv_path = '/content/drive/MyDrive/A2_Data.csv'
df = pd.read_csv(csv_path)

# Column name that contains the image URLs
image_column_name = 'Image'

# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.output)

def download_and_process_img(img_url):
    try:
        response = requests.get(img_url)
        img = Image.open(BytesIO(response.content))
        img = img.resize((224, 224))
        img_array = image.img_to_array(img)
        expanded_img_array = np.expand_dims(img_array, axis=0)
        return preprocess_input(expanded_img_array)
    except Exception as e:
        print(f"Error processing image {img_url}: {e}")
        return None

def extract_features(df, column_name):
    features = {}
    for index, row in df.iterrows():
        img_url = row[column_name].strip("[]''")  # Remove extra characters
        processed_img = download_and_process_img(img_url)
        if processed_img is not None:
            feature = model.predict(processed_img)
            features[img_url] = feature.flatten()
        else:
            print(f"Skipping image {img_url}")

    return features


# Extract features using the CSV column
image_features = extract_features(df, image_column_name)
output_path = '/content/drive/MyDrive/image_features.pkl'

# Save the features to a .pkl file
with open(output_path, 'wb') as f:
    pickle.dump(image_features, f)

print(f'Image features saved to {output_path}')


1/1 [==============================] - 0s 475ms/step
Error processing image https://images-na.ssl-images-amazon.com/images/I/71F3npeHUDL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71wHUWncMGL._SY88.jpg: cannot identify image file <_io.BytesIO object at 0x7c76238d2d40>
Skipping image https://images-na.ssl-images-amazon.com/images/I/71F3npeHUDL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71wHUWncMGL._SY88.jpg
1/1 [==============================] - 1s 521ms/step
Error processing image https://images-na.ssl-images-amazon.com/images/I/71B8OOE5N8L._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/81SX3oAWbNL._SY88.jpg: cannot identify image file <_io.BytesIO object at 0x7c7623c14310>
Skipping image https://images-na.ssl-images-amazon.com/images/I/71B8OOE5N8L._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/81SX3oAWbNL._SY88.jpg
1/1 [==============================] - 0s 481ms/step
Error processing image https://images-na.ssl-images

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import math
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
import pickle

# Assuming tfidf_docs contains TF-IDF scores for each document as calculated above



# Assuming nltk resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # Lowercase, remove punctuation, tokenize, remove stopwords, and lemmatize
    text = str(text).lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

def compute_tf(document):
    # Computes TF for each word in a document
    tf_dict = {}
    bow_count = len(document)
    word_counts = Counter(document)
    for word, count in word_counts.items():
        tf_dict[word] = count / float(bow_count)
    return tf_dict

def compute_idf(doc_list):
    # Create a set of all words across all documents
    all_words = set(word for doc in doc_list for word in doc)
    # Initializes IDF dict with zeros for each word
    idf_dict = dict.fromkeys(all_words, 0)
    N = len(doc_list)

    # Count documents containing each word
    for document in doc_list:
        for word in set(document):  # Use set to count each word once per document
            idf_dict[word] += 1

    # Calculate IDF for each word
    for word, val in idf_dict.items():
        idf_dict[word] = math.log(N / float(val))
    return idf_dict


def compute_tfidf(tf_bow, idfs):
    # Compute TF-IDF for each word in a document
    tfidf = {}
    for word, val in tf_bow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

# Example usage with your dataset
csv_path = '/content/drive/MyDrive/A2_Data.csv'  # Path to your CSV file
df = pd.read_csv(csv_path)
processed_docs = [preprocess_text(doc) for doc in df['Review Text']]  # Assuming 'Review' is the column

# Calculate TF for each document
tf_bows = [compute_tf(doc) for doc in processed_docs]

# Calculate IDF using all documents
idfs = compute_idf(tf_bows)

# Calculate TF-IDF for each document
tfidf_docs = [compute_tfidf(tf_bow, idfs) for tf_bow in tf_bows]

# Now, tfidf_docs contains TF-IDF scores for each document
# Specify the path where you want to save the TF-IDF scores
output_path = '/content/drive/MyDrive/tfidf_scores.pkl'

# Save the TF-IDF scores to a .pkl file
with open(output_path, 'wb') as f:
    pickle.dump(tfidf_docs, f)

print(f'TF-IDF scores saved to {output_path}')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


TF-IDF scores saved to /content/drive/MyDrive/tfidf_scores.pkl


In [ ]:
import pickle
import pandas as pd
import numpy as np
from collections import Counter
import math
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
import pickle
def cosine_similarity_manual(v1, v2):
    dot_product = sum(a*b for a, b in zip(v1, v2))
    magnitude_v1 = sum(a*a for a in v1) ** 0.5
    magnitude_v2 = sum(b*b for b in v2) ** 0.5
    if magnitude_v1 == 0 or magnitude_v2 == 0:
        return 0
    return dot_product / (magnitude_v1 * magnitude_v2)
# Load image features
with open('/content/drive/MyDrive/image_features.pkl', 'rb') as f:
  image_features = pickle.load(f)

# Example query image feature vector (use an actual vector from your dataset or a new image)
query_image_vector = np.array(list(image_features.values())[0])  # Using the first image as an example

# Calculate similarities
image_similarities = {img_url: cosine_similarity_manual(query_image_vector, features) for img_url, features in image_features.items()}

# Get top 3 similar images
top_3_images = sorted(image_similarities.items(), key=lambda item: item[1], reverse=True)[:3]
# Assume tfidf_docs is a list of dictionaries, each representing a document's TF-IDF vector
with open('/content/drive/MyDrive/tfidf_scores.pkl', 'rb') as f:
    tfidf_docs = pickle.load(f)

# Example query text TF-IDF vector (use an actual vector from your dataset or a new piece of text)
query_text_vector = np.array(list(tfidf_docs[0].values()))  # Using the first document as an example

# Calculate similarities
text_similarities = {i: cosine_similarity_manual(query_text_vector, np.array(list(doc.values()))) for i, doc in enumerate(tfidf_docs)}

# Get top 3 similar texts
top_3_texts = sorted(text_similarities.items(), key=lambda item: item[1], reverse=True)[:3]
with open('/content/drive/MyDrive/top_3_images.pkl', 'wb') as f:
    pickle.dump(top_3_images, f)

with open('/content/drive/MyDrive/top_3_texts.pkl', 'wb') as f:
    pickle.dump(top_3_texts, f)



In [ ]:
import pickle

# Load the top matches for images and texts
with open('/content/drive/MyDrive/top_3_images.pkl', 'rb') as f:
    top_images = pickle.load(f)
with open('/content/drive/MyDrive/top_3_texts.pkl', 'rb') as f:
    top_texts = pickle.load(f)

# Assuming the structure for top_images and top_texts is [(key, similarity_score), ...]
# Calculate composite similarity scores
composite_scores = []
for img_tuple, text_tuple in zip(top_images, top_texts):
    img_url, img_similarity = img_tuple
    text_index, text_similarity = text_tuple
    composite_score = (img_similarity + text_similarity) / 2
    composite_scores.append(((img_url, text_index), composite_score))

# Rank pairs based on the composite similarity score
ranked_pairs = sorted(composite_scores, key=lambda x: x[1], reverse=True)

# Save the ranked pairs
with open('/content/drive/MyDrive/ranked_pairs.pkl', 'wb') as f:
    pickle.dump(ranked_pairs, f)


In [ ]:
import pickle

# Load the ranked pairs
with open('/content/drive/MyDrive/ranked_pairs.pkl', 'rb') as f:
    ranked_pairs = pickle.load(f)

# Present the top-ranked pairs
print("Top-ranked (image, review) pairs with their composite cosine similarity scores:")
for i, ((img_url, text_index), score) in enumerate(ranked_pairs):
    print(f"{i+1}: Image URL: {img_url}, Review Index: {text_index}, Score: {score}")


Top-ranked (image, review) pairs with their composite cosine similarity scores:
1: Image URL: https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg, Review Index: 0, Score: 1.0
2: Image URL: https://images-na.ssl-images-amazon.com/images/I/71Y9QQZDpvL._SY88.jpg, Review Index: 353, Score: 0.6071899544871118
3: Image URL: https://images-na.ssl-images-amazon.com/images/I/712LuK8v2+L._SY88.jpg, Review Index: 880, Score: 0.6035028952618557


In [ ]:
import pandas as pd

def find_review_by_image(csv_path, image_url):
    # Load the CSV file
    df = pd.read_csv(csv_path)

    # Preprocess the 'Image' column to match the input format
    df['Image'] = df['Image'].str.strip("[]'")  # Strip the [' and '] characters

    # Find the row with the matching image URL
    matching_row = df[df['Image'] == image_url]

    if not matching_row.empty:
        return matching_row.iloc[0]['Review Text']
    else:
        return "No matching review found for the given image URL."

# User input
input_image_url = input("Please enter link of Image:")
csv_path = '/content/drive/MyDrive/A2_Data.csv'

# Find and print the matching review
review_text = find_review_by_image(csv_path, input_image_url)
print(review_text)




Please enter link of Image:https://images-na.ssl-images-amazon.com/images/I/71bztfqdg+L._SY88.jpg
I have been using Fender locking tuners for about five years on various strats and teles. Definitely helps with tuning stability and way faster to restring if there is a break.


In [ ]:
import pickle
import numpy as np

# Function to calculate cosine similarity
def cosine_similarity_manual(v1, v2):
    # Pad the shorter vector with zeros
    if len(v1) > len(v2):
        v2 = np.pad(v2, (0, len(v1) - len(v2)), 'constant')
    elif len(v2) > len(v1):
        v1 = np.pad(v1, (0, len(v2) - len(v1)), 'constant')

    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    if magnitude_v1 == 0 or magnitude_v2 == 0:
        return 0
    return dot_product / (magnitude_v1 * magnitude_v2)


# Load pre-computed image features and TF-IDF vectors
with open('/content/drive/MyDrive/image_features.pkl', 'rb') as f:
    image_features = pickle.load(f)
with open('/content/drive/MyDrive/tfidf_scores.pkl', 'rb') as f:
    tfidf_docs = pickle.load(f)

# Assuming you have lists of image URLs and review texts
# For this example, let's use placeholders
image_urls = ['https://images-na.ssl-images-amazon.com/images/I/71edIGOwydL._SY88.jpg']
review_texts=["i was expecting extreme brutality but got a pleasant surprise. this pup is super clean no. 1 and distortion is very smooth not at all what expected its warm in tone but has some bite to it. overall i really like this pup. to be honest I was expecting this pickup to be really mid focused but it has a tight punch in the low and the highs are not over bearing either. great pickup would get again!"]

# Convert TF-IDF dicts to vectors for comparison
tfidf_vectors = [np.array(list(doc.values())) for doc in tfidf_docs]

# Calculate cosine similarities
for img_url, review_text in zip(image_urls, review_texts):
    # Image similarity
    if img_url in image_features:
        img_vector = np.array(image_features[img_url])
        img_similarities = [cosine_similarity_manual(img_vector, features) for features in image_features.values()]
    else:
        print(f"Image URL {img_url} not found in pre-computed features.")
        continue

    # Text similarity
    text_vector = np.array(list(tfidf_docs[0].values()))  # Placeholder for actual text feature extraction
    text_similarities = [cosine_similarity_manual(text_vector, vec) for vec in tfidf_vectors]

    # Output
    print(f"Image URL: {img_url}")
    print(f"Review: {review_text}")
    print(f"Cosine similarity of images - {np.max(img_similarities):.4f}")
    print(f"Cosine similarity of text - {np.max(text_similarities):.4f}")


Image URL: https://images-na.ssl-images-amazon.com/images/I/71edIGOwydL._SY88.jpg
Review: i was expecting extreme brutality but got a pleasant surprise. this pup is super clean no. 1 and distortion is very smooth not at all what expected its warm in tone but has some bite to it. overall i really like this pup. to be honest I was expecting this pickup to be really mid focused but it has a tight punch in the low and the highs are not over bearing either. great pickup would get again!
Cosine similarity of images - 1.0000
Cosine similarity of text - 1.0000
